In [ ]:
# Importing the required libraries, custom libraries and setting the python path
from load_env_and_set_pythonpath import print_pythonpath
import os
import inversion_utils as iu
import helita_io_lp as lp
import time
import numpy as np
from skimage.transform import resize
from einops import rearrange
import ambig_utils as au
from sunpy.coordinates import sun
from astropy.coordinates import SkyCoord
import astropy.units as u
from sunpy.coordinates import frames
import numpy as np
if au.is_notebook():
    from IPython.display import clear_output
print_pythonpath()

In [ ]:
import importlib
importlib.reload(iu)

In [ ]:
# Load the configuration from the JSON file
input_config = iu.load_yaml_config('input_config.yaml')
# Check the input configuration
config = iu.check_input_config(input_config, pprint=True, confirm=False)

In [ ]:
# Load the configuration from the JSON file

# Path to ambig executable and par file
ambig_executable_path = config['ambig_executable_path']
ambig_par = config['ambig_par']

# Fcubes file names saved in save_dir directory
ambig_input_dir = config['ambig_input_dir']
fbazi = config['fbazi']
fbhor = config['fbhor']
fblos = config['fblos']
ambig_save_dir = config['ambig_save_dir']

# Cropping and resize details
crop = config['crop']
xorg = config['xorg']
xsize = config['xsize'] 
yorg = config['yorg']
ysize = config['ysize']
rescale = config['rescale']

# North up and field of view angle details
is_north_up = config['is_north_up']
fov_angle = config['fov_angle']

# Fits file details from CRISP header
fits_info = config['fits_info']
pix = config['scale']

# Verbosity in prints
ambig_verbose = config['ambig_verbose']

# Load the CRISP image info
crisp_im = config['crisp_im']
fits_info = config['fits_info']
all_start_times = fits_info['all_start_times']
fits_header = iu.load_fits_header(crisp_im, out_dict=False)
# time range for disabmiguation
ambig_time_range = config['ambig_time_range']
ambig_save_fits = config['ambig_save_fits']
ambig_save_lp = config['ambig_save_lp']
delete_ambig_temp_files = config['delete_ambig_temp_files']
if ambig_verbose:
    print('Ambiguity disambiguation time range: ', ambig_time_range[0], '-', ambig_time_range[-1])

In [ ]:
# Load the data from the saved files
blos_in = lp.getdata(os.path.join(ambig_input_dir, fblos)) # nx, ny, nt
bhor_in = lp.getdata(os.path.join(ambig_input_dir, fbhor))
bazi_in = lp.getdata(os.path.join(ambig_input_dir, fbazi))
print('blos shape:', blos_in.shape)

In [ ]:
temp_files_list = []

for tt in ambig_time_range:
    # Get the range of latitudes and longitudes from the fits file info
    x1 = fits_info['hplnt'][tt][0]
    x2 = fits_info['hplnt'][tt][1]
    y1 = fits_info['hpltt'][tt][0]
    y2 = fits_info['hpltt'][tt][1]
    xcen = (x1 + x2) / 2
    ycen = (y1 + y2) / 2
    tobs = fits_info['all_start_times'][tt]
    tstring = tobs.replace(':', '').replace(' ', '_T')

    if rescale and rescale != 1:
        prefix = f'{tstring}_scale_{rescale}'
    else:
        prefix = f'{tstring}'

    if blos_in.shape[2] == 1: # only one time step is loaded
    # drop the redundant time axis
        blos = blos_in.squeeze().T # ny, nx
        bhor = bhor_in.squeeze().T
        bazi = bazi_in.squeeze().T
    else:
        blos = blos_in[:,:,tt].T
        bhor = bhor_in[:,:,tt].T
        bazi = bazi_in[:,:,tt].T
    # Plot the data
    if ambig_verbose:
        iu.plot_image(blos, figsize=(6, 6), title='Blos')

    # Crop the data if needed
    if crop:
        blos = blos[yorg: yorg + ysize, xorg: xorg + xsize]
        bhor = bhor[yorg: yorg + ysize, xorg: xorg + xsize]
        bazi = bazi[yorg: yorg + ysize, xorg: xorg + xsize]
        if ambig_verbose:
            iu.plot_image(blos, figsize=(6, 6), title='Blos after cropping')

    if rescale and rescale != 1:
        ysize, xsize = blos.shape 
        ysize = ysize // rescale
        xsize = xsize // rescale
        if ysize % 2 != 0:
            ysize += 1
        if xsize % 2 != 0:
            xsize += 1
        if ambig_verbose:
            print(f'Shape before rescale: {blos.shape}')
        # Resample the array to the new size
        blos = resize(blos, (ysize, xsize), anti_aliasing=True)
        bhor = resize(bhor, (ysize, xsize), anti_aliasing=True)
        bazi = resize(bazi, (ysize, xsize), anti_aliasing=True)
        if ambig_verbose:
            print(f'Shape after rescale: {blos.shape}')
            iu.plot_image(blos, figsize=(6, 6), title='Blos after rescaling')

    # Rotate the data to make it north-up
    if not is_north_up:
        blos = iu.make_north_up(blos, fov_angle)
        bhor = iu.make_north_up(bhor, fov_angle)
        bazi = iu.make_north_up(bazi, fov_angle)
        ysize, xsize = blos.shape 
        if ambig_verbose:
            print(f'Shape after rotation: {blos.shape}')
        if ysize % 2 != 0 or xsize % 2 != 0:
            print('Resampling the array to ensure that the shape is a multiple of 2')
            print(f'Shape before resampling: {blos.shape}')
            # Resample the array to ensure that the shape is a multiple of 2
            ysize = ysize // 2 * 2
            xsize = xsize // 2 * 2
            blos = resize(blos, (ysize, xsize), anti_aliasing=True)
            bhor = resize(bhor, (ysize, xsize), anti_aliasing=True)
            bazi = resize(bazi, (ysize, xsize), anti_aliasing=True)
            if ambig_verbose:
                iu.plot_image(blos, figsize=(6, 6), title='Blos after rotation')

    # Create the mask for the data
    data_mask = blos != 0

    # Calculate the P, B0 and angular radius of the Sun, and convert the coordinates of center to Heliographic Stonyhurst longitude and latitude
    pbr = sun.P(tobs).value, sun.B0(tobs).value, sun.angular_radius(tobs).value
    coord = SkyCoord(xcen*u.arcsec, ycen*u.arcsec, obstime=tobs,
                    frame=frames.Helioprojective, observer='earth')
    coord_hs = coord.transform_to(frames.HeliographicStonyhurst)
    lonlat = coord_hs.lon.value, coord_hs.lat.value
    print(f'P, B0, angular radius: {pbr}')
    print(f'lon, lat: {lonlat}')

    for ii in range(1):
        print(f'Iteration: {ii}')
        # Write the ambig input file
        au.write_ambig_input('.', pix, pbr, lonlat, blos, bhor, bazi)
        # Run the ambig code
        bx, by, bz, bazi, metrics, fig = au.disambig_azimuth(bhor, blos, ambig_par, ambig_executable_path, prefix,
                                                              save_dir=ambig_save_dir, save_fig=True, data_mask=data_mask)
        # Read the parameters from the ambig par file
        params = au.read_ambig_par_file(ambig_par)
        # Update the seed value with iteration number
        params['iseed'] = ii
        # # Increase the tfactr value
        # params['tfactr'] *= 1.02
        au.write_ambig_par_file(ambig_par, params,pprint=True)
        # Clear the output if running in a notebook
        if au.is_notebook():
            clear_output()
    # remove the azimuth dat file
    azimuth_dat_copy = os.path.join(ambig_save_dir, prefix + 'azimuth.dat')
    if os.path.exists(azimuth_dat_copy):
        os.remove(azimuth_dat_copy)
    # Collect the results in a numpy array
    disambig_out = np.stack([bx, by, bz, bazi], axis=0)
    # Save the results in a fits file
    temp_fits = f'{ambig_save_dir}temp_{prefix}_t_{tt}_disambig_out.fits'
    iu.save_fits(disambig_out, fits_header, temp_fits, overwrite=True)
    print(f'Saved the disambiguated output in {temp_fits}')
    temp_files_list.append(temp_fits)

In [ ]:
var_list = ['bx', 'by', 'bz', 'bazi']
nt = len(temp_files_list)
nx = bz.shape[1]
ny = bz.shape[0]

time_index_range = f"{ambig_time_range[0]}-{ambig_time_range[-1]}"
obs_start_time = all_start_times[ambig_time_range[0]].replace(':', '').replace(' ', '_T')
obs_end_time = all_start_times[ambig_time_range[-1]].replace(':', '').replace(' ', '_T')

for var in var_list:
    # create a numpy array with nx, ny , nt dimensions
    full_var_data = np.zeros((nt, nx, ny))
    var_index = var_list.index(var)
    sav_var = var_list[var_index].capitalize()

    for ii in range(len(ambig_time_range)):
        tt = ambig_time_range[ii]
        time_string = all_start_times[tt].replace(':', '').replace(' ', '_T')
        out_file_name = temp_files_list[ii]
        var_data = iu.load_fits_data(out_file_name)
        var_header = iu.load_fits_header(out_file_name, out_dict=False)
        full_var_data[ii] = var_data[var_index].T
        
    if ambig_save_fits:
        # Save the full variable data
        if rescale and rescale != 1:
            out_file_name = os.path.join(ambig_save_dir, f'{sav_var}_{obs_start_time}_{obs_end_time}_t_{time_index_range}_s_{rescale}.fits')
        else:
            out_file_name = os.path.join(ambig_save_dir, f'{sav_var}_{obs_start_time}_{obs_end_time}_t_{time_index_range}.fits')
        iu.save_fits(full_var_data, var_header, out_file_name, overwrite=True, verbose=ambig_verbose)

    if ambig_save_lp:
        # Save the inversion output in the LP format
        if rescale and rescale != 1:
            lp_out_file_name = os.path.join(ambig_save_dir, f'{sav_var}_{obs_start_time}_{obs_end_time}_t_{time_index_range}_s_{rescale}.fcube')
        else:
            lp_out_file_name = os.path.join(ambig_save_dir, f'{sav_var}_{obs_start_time}_{obs_end_time}_t_{time_index_range}.fcube')
        lp_data = np.float32(rearrange(full_var_data, 'nt nx ny -> nx ny nt'))
        lp.writeto(lp_out_file_name, lp_data, extraheader='', dtype=None, verbose=True, append=False)
    
if delete_ambig_temp_files:
    # Delete the temporary file using os module
    for temp_file in temp_files_list:
        if ambig_verbose:
            print(f'Deleting temporary file: {temp_file}')
        os.remove(temp_file)

# copy the ambig_par file to the save_dir with suffix appended
ambig_par_copy = ambig_save_dir  + prefix + '_par'
os.system(f'cp {ambig_par} {ambig_par_copy}')
if ambig_verbose:
    print(f'Copied {ambig_par} to {ambig_par_copy}')